# Reacting to events

If a friend has high blood pressure we better let them know!

We'll need some stuff prom those previous notebooks...

In [34]:
#Hour and minute when this started to create unique names
%store -r unique
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A sample message payload
%store -r payload

In this example we are going to emmit a simple print to the logs.

This is your code and could do anything: send an em-mail, a tweet, a phone call...

In [35]:
# %load hypertension.py
def on_hbp(event, context):
    name = event['name']
    print("High blood pressure detected. Take care {}!".format(name))


To create a lambda function we need to:

- Create a zip package with the code

In [36]:
zip_file = "{}/aws_iot/hypertension{}.zip".format(home,unique) 
!zip -g {zip_file} hypertension.py
zip_file

	zip warning: /home/ec2-user/aws_iot/hypertensionehw3758579571.zip not found or empty
  adding: hypertension.py (deflated 11%)


'/home/ec2-user/aws_iot/hypertensionehw3758579571.zip'

- Create a role authorizing the function execution

In [37]:
print(open('lambda-trust.json').read())

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}


In [38]:
role_name = "fn_hbp_role_{}".format(unique) 
fn_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://lambda-trust.json \
    --query Role.Arn \
    --output text
fn_role_arn = fn_role_arn.s
fn_role_arn

'arn:aws:iam::030555009967:role/fn_hbp_role_ehw3758579571'

In [39]:
policy_arn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
!aws iam attach-role-policy \
    --role-name '{role_name}' \
    --policy-arn '{policy_arn}'

In [40]:
!echo {fn_role_arn}

arn:aws:iam::030555009967:role/fn_hbp_role_ehw3758579571


Create the function:

In [41]:
function_name="on_hbp_{}".format(unique)
function_name

'on_hbp_ehw3758579571'

In [42]:
#Wait for it
import time
time.sleep(10)

In [43]:
function_arn = !aws lambda create-function \
    --function-name '{function_name}' \
    --zip-file "fileb://{zip_file}" \
    --role '{fn_role_arn}' \
    --handler hypertension.on_hbp \
    --runtime python3.6 \
    --timeout 30 \
    --memory-size 128 \
    --query FunctionArn \
    --output text
function_arn = function_arn.s
function_arn

'arn:aws:lambda:us-east-1:030555009967:function:on_hbp_ehw3758579571'

Now let's create the rule that tells AWS IoT to call that function when a message with high blood pressure arrives.

For that we need:
- The SQL statement capturing messages with the desired condition

In [44]:
sql = """
SELECT name, 
    time_stamp, 
    systolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{}'
WHERE systolic > 120 
    OR diastolic > 80
""".format(topic_name)
print(sql)


SELECT name, 
    time_stamp, 
    systolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM 'bp_topic/data'
WHERE systolic > 120 
    OR diastolic > 80



- Authorize the rule to invoke the lambda function

https://docs.aws.amazon.com/iot/latest/developerguide/lambda-rule.html

In [45]:
import json
rule = {
        "sql": sql, 
        "ruleDisabled": False,
        "awsIotSqlVersion": "2016-03-23",
        "actions": [{
            "lambda": {
                "functionArn": function_arn
             }
        }]}
rule = json.dumps(rule, indent=4)
rule_file="{}/rule.json".format(home)
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

'/home/ec2-user/rule.json'

In [46]:
print(open(rule_file).read())

{
    "sql": "\nSELECT name, \n    time_stamp, \n    systolic, \n    diastolic, \n    id, \n    timestamp() as recv_stamp \nFROM 'bp_topic/data'\nWHERE systolic > 120 \n    OR diastolic > 80\n",
    "ruleDisabled": false,
    "awsIotSqlVersion": "2016-03-23",
    "actions": [
        {
            "lambda": {
                "functionArn": "arn:aws:lambda:us-east-1:030555009967:function:on_hbp_ehw3758579571"
            }
        }
    ]
}


In [47]:
rule_name="bp2lambda_rule_{}".format(unique)
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}
rule_name

'bp2lambda_rule_ehw3758579571'

In [48]:
rule_arn=!aws iot get-topic-rule --rule-name '{rule_name}' --query ruleArn --output text
rule_arn = rule_arn.s
rule_arn

'arn:aws:iot:us-east-1:030555009967:rule/bp2lambda_rule_ehw3758579571'

In [49]:
account_id=!aws sts get-caller-identity --query Account --output text
account_id =account_id.s
account_id

'030555009967'

In [50]:
unique_id = "statement_{}".format(unique)
unique_id

'statement_ehw3758579571'

In [51]:
!aws lambda add-permission \
    --function-name "{function_name}" \
    --principal iot.amazonaws.com \
    --source-arn '{rule_arn}' \
    --source-account "{account_id}" \
    --statement-id "{unique_id}" \
    --action "lambda:InvokeFunction"

{
    "Statement": "{\"Sid\":\"statement_ehw3758579571\",\"Effect\":\"Allow\",\"Principal\":{\"Service\":\"iot.amazonaws.com\"},\"Action\":\"lambda:InvokeFunction\",\"Resource\":\"arn:aws:lambda:us-east-1:030555009967:function:on_hbp_ehw3758579571\",\"Condition\":{\"StringEquals\":{\"AWS:SourceAccount\":\"030555009967\"},\"ArnLike\":{\"AWS:SourceArn\":\"arn:aws:iot:us-east-1:030555009967:rule/bp2lambda_rule_ehw3758579571\"}}}"
}


# Triggering the rule

In [52]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="bp_client_{}".format(unique) 
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(600)
mqtt.connect()
mqtt

In [53]:
data = {"id": "fa91d2de-24a1-4052-8b58-4f2ab743f20k",
        "name": "Rose Tyle", 
        "systolic": 130, 
        "diastolic": 90, 
        "time_stamp": 1541059348245}
import json
msg = json.dumps(data, indent=4)
mqtt.publish(topic_name, msg , QoS = 0)
print(msg)

{
    "id": "fa91d2de-24a1-4052-8b58-4f2ab743f20k",
    "name": "Rose Tyle",
    "systolic": 130,
    "diastolic": 90,
    "time_stamp": 1541059348245
}


In [54]:
#Wait for it
import time
time.sleep(5)

In [55]:
log_group="/aws/lambda/{}".format(function_name)
print(log_group)
!aws logs describe-log-groups --log-group-name-prefix '{log_group}'

/aws/lambda/on_hbp_ehw3758579571
{
    "logGroups": [
        {
            "logGroupName": "/aws/lambda/on_hbp_ehw3758579571",
            "creationTime": 1541151561677,
            "metricFilterCount": 0,
            "arn": "arn:aws:logs:us-east-1:030555009967:log-group:/aws/lambda/on_hbp_ehw3758579571:*",
            "storedBytes": 0
        }
    ]
}


In [68]:
log_stream = !aws logs describe-log-streams \
    --log-group-name '{log_group}' \
    --order-by LastEventTime \
    --max-items 1  \
    --query 'logStreams[0].logStreamName'\
    --output text
log_stream = log_stream.s
log_stream

'2018/11/01/[$LATEST]564ffab1fae0421084809accfb64526d'

Check for our messages in the logs:

In [58]:
!aws logs get-log-events \
    --log-group-name '{log_group}' \
    --log-stream-name '{log_stream}' \
    --query 'events[].message' \
    --output text 


An error occurred (InvalidParameterException) when calling the GetLogEvents operation: 1 validation error detected: Value '{log_group}' at 'logGroupName' failed to satisfy constraint: Member must satisfy regular expression pattern: [\.\-_/#A-Za-z0-9]+


In [59]:
mqtt.disconnect()

True

Thank you! That is it for now, time to build \o/

Keep updated with new releases of this workshop at https://internetof.healthcare